In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
plt.style.use('default')

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

from detection.dataset import VideoDataset, MyConcatDataset, VideoDatasetRNN
from detection.models import TrackNetV2MSE, TrackNetV2NLL, TrackNetV2RNN
from detection.training import train_model
from detection.testing import get_local_maxima

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

%load_ext autoreload
%autoreload 2

# Trajectory analysis with RANSAC on heatmaps

In [ ]:
from detection.testing import get_local_maxima

# heatmaps_folder = './checkpoints/tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_results/heatmaps_test_standard'
heatmaps_folder = './checkpoints/tracknet_v2_mse_360_640/checkpoint_0027_results/heatmaps_test_standard'

frame_range = np.arange(1200)

w_heatmap, h_heatmap = 640, 360

heatmaps = np.zeros((len(frame_range), h_heatmap, w_heatmap))
for k_seed, frame_index in enumerate(frame_range):
    heatmaps[k_seed] = cv2.imread(os.path.join(heatmaps_folder, f"{frame_index}".zfill(6)+'.png'), cv2.IMREAD_GRAYSCALE)/255

cap = cv2.VideoCapture("../datasets/test_standard/video.mp4")
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_range[0]+5)

# w_frame, h_frame = 640, 360
w_frame, h_frame = 1280, 720

frames = np.zeros((len(frame_range), h_frame, w_frame, 3), dtype=np.uint8)
for k_seed, frame_index in enumerate(frame_range):
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break
    frames[k_seed] = cv2.cvtColor(cv2.resize(frame, (w_frame, h_frame)), cv2.COLOR_BGR2RGB)
cap.release()

frt = w_frame/w_heatmap # ratio between frame and heatmap

In [ ]:
max_num_candidates = 20                # maximum supported number of candidates for each frame
n_candidates = np.zeros(len(heatmaps), dtype=int) # number of candidates for each frame

candidates = np.zeros((len(heatmaps), max_num_candidates, 2))  # candidates array
for k_seed in range(len(heatmaps)):
    m = get_local_maxima(heatmaps[k_seed], threshold=0.1, sigma=0.5, include_diagonal=True)
    candidates[k_seed, :len(m)] = m[:max_num_candidates]
    n_candidates[k_seed] = min(len(m), max_num_candidates)

In [ ]:
k_seed = 214
plt.imshow(frames[k_seed])
plt.imshow(cv2.resize(heatmaps[k_seed], (w_frame, h_frame)), cmap='gray', alpha=0.7)
plt.scatter(candidates[k_seed][:,1]*frt, candidates[k_seed][:,0]*frt, s=4, c='r')

plt.show()

## Fit model

In [ ]:
from trajectories.fitting import *
from trajectories.filtering import *
from trajectories.visualization import *
from trajectories.utils import Graph

In [ ]:
trajectory_info = []

info_keys = ['k_seed','k_min','k_mid','k_max','i_seed','i_min','i_mid','i_max','n_support','iterations']
l2i = {key: i for i, key in enumerate(info_keys)}

display_fit = False

t = time.time()

# for k in [2, 28, 43]:
# for k in range(61, 62):
for k in range(1200):
    k_seed = k

    d_threshold = 3
    seed_radius = 20

    N = 10

    parameters, info, trajectories, supports, costs = fit_trajectories(candidates, n_candidates, k_seed, seed_radius, d_threshold, N)

    if costs is None:
        if display_fit:
            print(f"Seed frame: {k}")
            plt.imshow(frames[k])
            plt.show()
        continue

    s = np.argmin(costs)

    if (parameters[s] == 0).all() or info[s,l2i['n_support']] < 5:
        # exclusion criteria:
        # - points
        # - support set smaller than 5
        continue

    if display_fit:
        # k_min = supports[s,0,]
        print(f"Found {len(costs)} seed triplet(s)")
        print(f"Seed frame: {info[s,l2i['k_seed']]}")
        print(f"Used frame for fit: {info[s,l2i['k_min']]}, {info[s,l2i['k_mid']]}, {info[s,l2i['k_max']]}")
        print(f"Used candidate for fit: {info[s,l2i['i_min']]}, {info[s,l2i['i_mid']]}, {info[s,l2i['i_max']]}")
        print(f"support: {supports[s]}")
        print(f"cost = {costs[s]}")
        print(f"Iterations: {info[s,l2i['iterations']]}")
        print(f"v = {parameters[s,0]}")
        print(f"a = {parameters[s,1]}")
        show_fit(trajectories[s], candidates,
                *[info[s, l2i[key]] for key in info_keys[:-2]],
                background=frames[info[s,l2i['k_min']]])
        plt.show()

    d = {key: info[s, l2i[key]] for key in info_keys}
    d['trajectory'] = trajectories[s]
    d['support'] = supports[s]
    d['v'] = parameters[s,0]
    d['a'] = parameters[s,1]
    d['cost'] = costs[s]
    trajectory_info.append(d)

t1 = time.time()
print(f"Execution time: {t1-t}")

plt.show()

In [ ]:
def extract(trajectory_info, i):
    k_seed = trajectory_info[i]['k_seed']
    trajectory = trajectory_info[i]['trajectory']
    n_support = trajectory_info[i]['n_support']
    support = trajectory_info[i]['support'][:n_support]
    return trajectory, support, k_seed

In [ ]:
from networkx import DiGraph, dijkstra_path

max_diff = N

trajectory_graph = DiGraph()

for i in range(len(trajectory_info)):
    for j in range(i, min(i+max_diff, len(trajectory_info))):
        t1, s1, k1 = extract(trajectory_info, i)
        t2, s2, k2 = extract(trajectory_info, j)
        d = trajectory_distance(t1, s1, k1, t2, s2, k2)

        if d != np.inf and i!=j:
            trajectory_graph.add_edge(k1, k2, weight=d)

In [ ]:
shortest_path = dijkstra_path(trajectory_graph, 1, 108)
shortest_path

In [ ]:
t1, s1, k1 = extract(trajectory_info, 107)
t2, s2, k2 = extract(trajectory_info, 108)
d = trajectory_distance(t1, s1, k1, t2, s2, k2)
d

In [ ]:
ti = np.array([t['k_seed'] for t in trajectory_info])
trajectory_info[np.where(ti==137)[0][0]]

In [ ]:
np.where(ti==199)[0][0]

In [ ]:
ti = np.array([t['k_seed'] for t in trajectory_info])

dpi = 100
fig, ax = plt.subplots(figsize=(w_frame/dpi, h_frame/dpi), dpi=dpi)

ax.imshow(frames[91], zorder=-2)

colors = ['r', 'g', 'y', 'b']

to_print = []
for i, node in enumerate(shortest_path):
    if node not in range(60, 85): # (70, 80, 83, 89, 87):
        continue

    to_print.append(node)
    index = np.where(ti==node)[0][0]
    trajectory = trajectory_info[index]['trajectory']

    show_fit(trajectory*frt, candidates*frt, *[trajectory_info[index][key] for key in info_keys[:-2]],
             ax=ax,
             annotate=True,
             show_fitting_points=False,
             trajectory_color=colors[i%len(colors)])# 'y' if i%2==0 else 'r')
ax.set_xlim(0, w_frame)
ax.set_ylim(h_frame, 0)

print(to_print)
plt.show()

In [ ]:
trajectory_info[0].keys()

In [ ]:
index = np.where(ti==99)[0][0]
trajectory = trajectory_info[index]['trajectory']
trajectory_info[index]['k_min']

In [ ]:
w_frame, h_frame = 1280, 720

In [ ]:
ti = np.array([t['k_seed'] for t in trajectory_info])

dpi = 100
fig, ax = plt.subplots(figsize=(w_frame/dpi, h_frame/dpi), dpi=dpi)

for frame in (107, 108):

    # frame = 108
    index = np.where(ti==frame)[0][0]
    trajectory = trajectory_info[index]['trajectory']

    show_fit(trajectory*frt, candidates*frt, *[trajectory_info[index][key] for key in info_keys[:-2]],
                ax=ax,
                annotate=True,
                show_fitting_points=True,
                background=frames[frame])# 'y' if i%2==0 else 'r')
    ax.set_xlim(0, w_frame)
    ax.set_ylim(h_frame, 0)

plt.show()

# Visualize some activations and kernels because why not

In [ ]:
model = TrackNetV2RNN(sequence_length=4)
model.load('checkpoints/tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_best.ckpt')
model.eval()
model

In [ ]:
dataset_params = dict(image_size=(360, 640),
                      sequence_length=4,
                      sigma=5,
                      drop_duplicate_frames=False,
                      transform = ToTensor(),
                      target_transform = ToTensor(),
                      grayscale=False)

dataset = VideoDatasetRNN(root="../datasets/prova/", **dataset_params)

In [ ]:
counter = 0

def get_encoding_layer(desired_block=1, subblock=0):
    layers = []
    for i, block in enumerate(model.children()):
        # print(i)
        if i%2 == 1:
            layers.append(block)
        for j, block_element in enumerate(block.children()):
            #print(i, j)
            for k, layer in enumerate(block_element.children()):
                layers.append(layer)
                # print(i, j, k)
                if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                    break
            if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                break
        if type(layer) is torch.nn.ReLU and i==2*desired_block:
            break
    return layers

def compute_activations(layers, input):
    activation = input.unsqueeze(dim=0)
    with torch.no_grad():
        for l in layers:
            activation = l(activation)

    return activation.squeeze().numpy()

In [ ]:
frames, labels = dataset[50]
frames = frames.to(torch.float32)

In [ ]:
w, h_frame, dpi = 300*2*16/9, 300, 100

fig, axs = plt.subplots(ncols=2, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

axs[0].imshow(frames[-3:].numpy().transpose(1, 2, 0))
axs[0].set_title("Input frame (last in sequence)")

axs[1].imshow(labels[0])
axs[1].set_title("Ground truth")

fig.tight_layout(pad=0.2)
plt.show()

In [ ]:
noise_part = np.linspace(0, 1, 10)
c = []

for n in noise_part:
    with torch.no_grad():
        f = (1-n)*frames + n*torch.randn(frames.shape)
        out = model(f.unsqueeze(dim=0)).squeeze().numpy()
    c.append(out.max())
plt.plot(noise_part, c)

In [ ]:
n = 0.07
with torch.no_grad():
    f = (1-n)*frames + n*torch.randn(frames.shape)
    out = model(f.unsqueeze(dim=0)).squeeze().numpy()
plt.imshow(out)
plt.colorbar()
plt.show()

In [ ]:
frames[:3] = torch.zeros(3, 360, 640)

In [ ]:
block = 2
subblock = 1

activations = compute_activations(get_encoding_layer(block, subblock), frames)
activations.shape

In [ ]:
(dead_activations, ) = np.where(activations.max(axis=(1,2))==0)
print(f"Of {activations.shape[0]} activations, {dead_activations.size} are dead and {activations.shape[0]-dead_activations.size} are not.")

In [ ]:
height_pixels = 1080
top_adjust = 1

w, h_frame, dpi = height_pixels*16/9*top_adjust, height_pixels, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

i_0 = 0

for k, ax in enumerate(axs.ravel()):
    ax.imshow(activations[k+i_0], cmap='gray')
    # ax.set_title(i)
    ax.set_axis_off()

#fig.suptitle(f"Activations in encoding block {block}, subblock {subblock}")

fig.tight_layout(pad=0.5)
fig.subplots_adjust(top=top_adjust)

#fig.savefig(f"{block}_{subblock}.png")

plt.show()

In [ ]:
model.state_dict().keys()

In [ ]:
dk = 4

kernels = model.state_dict()['vgg_conv1.1.0.weight'].numpy()
biases = model.state_dict()['vgg_conv1.1.0.bias'].numpy()
w, h_frame, dpi = 800, 800, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

print(kernels.shape)
print(biases[dk])

min_val = kernels[dk].min()
max_val = kernels[dk].max()
print(min_val, max_val)

max_val=max((max_val, -min_val))
min_val=min((-max_val, min_val))

for k, ax in enumerate(axs.ravel()):
    ax.imshow(kernels[dk,k], cmap='RdBu', vmin=min_val, vmax=max_val)
    ax.set_axis_off()

#fig.suptitle(f"Kernel {k}, bias = {biases[k]:.2g}")
fig.tight_layout(pad=0.2)
plt.show()
